# Tarea PySpark

### Objetivo:
 Analizar la eficiencia de los jugadores en términos generales y por posición, así como determinar la contribución al equipo por jugador tomando en cuenta los datos obtenidos

Usarás la base de datos del archivo 'fusbol.csv' para obtener tus datos. Checa la estructura del archivo para ver si es necesario limpiar la informacion, ver su estructura y así sea más fácil completar la tarea. Besos.

### Ejercicio 1:
Carga la base de datos en un DataFrame de Pyspark (con 2 nucleos). Valida los rangos de los valores donde sea aplicable, así como su corrección (en caso de ser necesaria). 
Después, utilizando las variables más relevantes como 'Ast/90', 'PassCmp%', etc., concluye qué ligas tienen los mejores jugadores por posición. Es decir, si los mejores jugadores defensas son de la liga francesa, inglesa, etc., por ejemplo.
Como cada persona tiene una definición de "mejor", utiliza las siguientes metricas por posición:
- Delanteros: npG+A/90 y npxG+xA/90
- Medios: KeyPass/90 y PassCmp%
- Defensas: PressSucc% y Interceptions/90

Con los resultados obtenidos, grafica por posición para que tu conclusión tenga un respaldo visual también.

In [21]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, avg, percent_rank
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import numpy as np

In [25]:
# Cargar los datos desde un archivo CSV
df = spark.read.csv('fusbol.csv', header=True, inferSchema=True)

# Mostrar las primeras filas y el esquema del DataFrame para entender la estructura de los datos
df.show(5)
df.printSchema()

# Limpiar y validar los datos si es necesario (asumiendo validaciones hipotéticas)
df = df.withColumn("Ast/90", when(col("Ast/90") < 0, 0).otherwise(col("Ast/90")))
df = df.withColumn("PassCmp%", when(col("PassCmp%") > 100, 100).otherwise(col("PassCmp%")))

NameError: name 'spark' is not defined

In [ ]:
# Filtrar delanteros y calcular métricas relevantes
delanteros = df.filter(col("Pos") == "Forward")
delanteros = delanteros.withColumn("npG+A/90", col("npG/90") + col("Ast/90"))
delanteros = delanteros.withColumn("npxG+xA/90", col("npxG/90") + col("xA/90"))

# Agregar la liga como una dimensión de análisis
mejores_delanteros = delanteros.groupBy("Squad").agg(
    _sum("npG+A/90").alias("Total npG+A/90"),
    _sum("npxG+xA/90").alias("Total npxG+xA/90")
).orderBy("Total npG+A/90", ascending=False)

mejores_delanteros.show()

In [ ]:
# Filtrar medios y calcular métricas relevantes
medios = df.filter(col("Pos").contains("MF"))
medios = medios.withColumn("KeyPass/90", col("KeyPass/90"))
medios = medios.withColumn("PassCmp%", col("PassCmp%"))

# Agregar la liga como una dimensión de análisis
mejores_medios = medios.groupBy("Comp").agg(
    _sum("KeyPass/90").alias("Total KeyPass/90"),
    _sum("PassCmp%").alias("Total PassCmp%")
).orderBy("Total KeyPass/90", ascending=False)

mejores_medios.show()

In [ ]:
# Filtrar defensas y calcular métricas relevantes
defensas = df.filter(col("Pos").contains("DF"))
defensas = defensas.withColumn("Interceptions/90", col("Interceptions/90"))

# Agregar la liga como una dimensión de análisis
mejores_defensas = defensas.groupBy("Comp").agg(
    _sum("Interceptions/90").alias("Total Interceptions/90")
).orderBy("Total Interceptions/90", ascending=False)

mejores_defensas.show()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Iniciar una sesión de Spark
spark = SparkSession.builder.master("local[2]").appName("Analisis de Futbol").getOrCreate()

# Cargar los datos desde un archivo CSV
df = spark.read.csv('fusbol.csv', header=True, inferSchema=True)

# Contar cuántos jugadores hay por posición
positions_count = df.groupBy("Pos").count()

In [ ]:
import matplotlib.pyplot as plt

# Convertir a pandas DataFrame para graficar
positions_count_pd = positions_count.toPandas()

# Crear un gráfico de barras
plt.figure(figsize=(12, 6))
plt.bar(positions_count_pd['Pos'], positions_count_pd['count'], color='skyblue')
plt.xlabel('Posición')
plt.ylabel('Número de Jugadores')
plt.title('Cantidad de Jugadores por Posición')
plt.xticks(rotation=45)
plt.show()

### Ejercicio 2:

Ahora hagamos algo un poco más interesante. Escoge algún jugador de todos los disponibles y toma 5 metricas, las que quieras. Debes concluir en qué percentil se encuentra el jugador en esas métricas que escogiste. Obviamente, vas a comparar sus valores con todos los demás con los que comparte posición y liga, para no tener un sesgo y que la información no pierda robustez. Por último, genera un DataFrame de Pyspark con todos los datos solicitados. Muestra el DataFrame y conviertelo a otro de tipo pandas. Muestra los dos.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, percent_rank, monotonically_increasing_id
from pyspark.sql.window import Window

# Iniciar sesión de Spark
spark = SparkSession.builder.master("local[2]").appName("Analisis de Jugadores").getOrCreate()

# Cargar el DataFrame
df = spark.read.csv('fusbol.csv', header=True, inferSchema=True)

# Elegir un jugador aleatorio para el ejemplo, supongamos que el primero
jugador = df.first()
# Métricas seleccionadas: 'Shots/90', 'Interceptions/90', 'PassCmp%', 'Ast/90', 'Touches/90'
metricas = ['Shots/90', 'Interceptions/90', 'PassCmp%', 'Ast/90', 'Touches/90']
jugador_pos = jugador['Pos']
jugador_liga = jugador['Comp']

# Filtrar jugadores de la misma posición y liga
df_filtrado = df.filter((col('Pos') == jugador_pos) & (col('Comp') == jugador_liga))

# Calcular los percentiles para cada métrica
window = Window.partitionBy("Pos", "Comp").orderBy([col(m) for m in metricas])
percentiles = df_filtrado.withColumn("index", monotonically_increasing_id()).select(
    "Player",
    *[col(m) for m in metricas],
    *[percent_rank().over(window).alias(m + '_percentile') for m in metricas]
).filter(col("index") == 0)  # Suponemos que es el primer jugador

# Mostrar los resultados
percentiles.show()
percentiles_pd = percentiles.toPandas()
percentiles_pd